
---


**Projet Dapyness - Estimation de ventes e-commerce** 


---

*L’objectif de ce projet est d’estimer l’évolution du volume de vente de produits vendus en ligne par un site e-commerce en utilisant les données de ces ventes.*

*Pour cela, nous étudions une table de données que nous nommons **df** lors de l'importation.*



*   Nous commençons notre étude par importer, analyser et nettoyer notre jeu de données. 
*   Ensuite, nous créons de nouvelles variables jugées utiles pour la suite de notre étude.
*   Nous analysons ensuite graphiquement nos données.
*   Et enfin, nous procédons à la modélisation de nos données pour espérer obtenir de bonnes prévisions de ventes.






L'ensemble des fichiers codes de ce projet sont décomposés en 5 parties :



*   01 - Analyse exploratoire 1/2 - Etude des différentes colonnes
*   02 - Analyse exploratoire 2/2 - Etude des valeurs manquantes
*   **03 - Création de nouvelles colonnes**
*   04 - Etude graphique des variables et tests statistiques
*   05 - Modélisation - Time Series

# **Préparation du Dataset et premières informations**

In [ ]:
# Cellule d'imporation des packages nécessaires aux codes

#packages basiques
import pandas as pd
import numpy as np

#partie graphique
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

from bokeh.plotting import figure, output_notebook, show
output_notebook()

#tests statistiques
import statsmodels.api 
from scipy.stats import pearsonr

#modélisation
#régression
from sklearn import model_selection
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.metrics import mean_squared_error

#modèles arima et sarima
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_pacf,plot_acf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX



In [ ]:
#Importation du fichier csv chez Sanae 
df=pd.read_csv('data-estimation.csv', sep=';')

In [ ]:
#Remplacement des valeurs manquantes NaN de CustomerID par une catégorie "Client non identifié"
df['CustomerID'].fillna('Client non identifié', inplace =True)

In [ ]:
#Suppression des lignes contenant une valeur manquante NaN dans la variable Description 
df= df.dropna(axis=0,how='any', subset=['Description'])

# **Création de nouvelles colonnes**

Dans cette section, nous créons de nouvelles variables : Date, Year, Month, Day, DayOfWeek, DayName, Week, Hour et InvoiceType

**Création d'une nouvelle colonne TotalAmount représentant le montant total par ligne (Quantité * Prix)**

In [ ]:
#création d'une nouvelle colonne TotalAmount 
df['TotalAmount'] = df['Quantity'] * df['UnitPrice']

**Rendre exploitable et analysable les données de la variable 'InvoiceDate', en plusieurs étapes, à l'heure de la méthode .to_datetime**

In [ ]:
#Conversion de la colonne InvoiceDate en to_datetime (car initialement de type object)
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [ ]:
### Rendre analysable les données de la variable 'InvoiceDate' 

#Exemple avec la colonne 'Year'

## Ajout d'une colonne qui stocke les données qui correspondent à
## l'année de facturation issues de la colonne InvoiceDate de df 

# transformer les données de InvoiceDate en série temporelle
# avec "pd.to_datetime(df.InvoiceDate)"
# transformer les données de InvoiceDate en datatimeproperties
# avec ".dt"
# en utilisant l'attribut ".year"
# stocker ce que ce code renvoie dans une nouvelle colonne de df
# nommée 'Year'

In [ ]:
#colonne correspondant à la date (sans la partie timing)
df['Date'] = pd.to_datetime(df.InvoiceDate).dt.date
#année, mois et jour de mois et de la semaine
df['Year'] = pd.to_datetime(df.InvoiceDate).dt.year
df['Month'] = pd.to_datetime(df.InvoiceDate).dt.month
df['Day'] = pd.to_datetime(df.InvoiceDate).dt.day
df['DayOfWeek'] = pd.to_datetime(df.InvoiceDate).dt.weekday
df['DayName'] = df.InvoiceDate.dt.day_name()
#numéro de la semaine de l'année
df['Week']=df.InvoiceDate.dt.week
#création d'une colonne heure car time est trop précis
df['Hour'] = pd.to_datetime(df.InvoiceDate).dt.hour

**Création d'une nouvelle colonne InvoiceType**



---
Nous décidons de créer cette colonne qui nous permettra de différencier les ventes, les retours (ou commandes annulées), les frais d'envoi, frais de transport, frais amazon, frais bancaires, dettes, commission, échantillons, et cadeaux ou problème de commande.

In [ ]:
''' Méthodologie '''
''' 

0) Nous commençons par créer notre nouvelle variable InvoiceType en y attribuant une valeur par défaut 'abcd'

1) Dans un premier temps nous remplaçons les valeurs 'abcd' selon certaines conditions : 
   --> Si la facture est composée de la lettre A ==> on remplace 'abcd' par 'Debt'
   --> Si la facture est composée de la lettre C ==> on remplace 'abcd' par 'Canceled'
   --> Si la facture n'est pas composé d'une lettre ==> on remplace 'abcd' par 'Sale' 

2) Cependant, certaines lignes que nous avons catégorisés en 'Canceled' et 'Sale' correspondent à 
d'autres opérations comme des frais, échantillons, commission, etc... 
   --> On utilise donc la colonne StockCode qui nous permet de détecter ces différentes opérations et on modifie 
       la valeur 'Canceled' et/ou 'Sale' par une autre catégorie correspondant à l'opération en question.

3) Lorsque le client est non identifié + prix = 0 + quantity < 0 ==> nous avons des descriptions anormale
qui correspondent plutôt à des problèmes de commande 
   --> On change donc les modalités 'Cancelled' et 'Sale' et on catégorise ces lignes en 'Order problem'

4) On retrouve quelques lignes pour lesquelles le prix est nul -> on considère que cela correspond à des 
cadeaux ou offres
   --> On change donc les modalités 'Cancelled' et 'Sale' et on catégorise ces lignes en 'Gift' '''

In [ ]:
# 0) création de la nouvelle colonne InvoiceType avec valeur par défaut 'abcd'
df['InvoiceType']='abcd'
df.head()

In [ ]:
# 1) 

#Catégorisation des dettes dans notre nouvelle colonne InvoiceType
df[df['InvoiceNo'].str.contains("A")]=df[df['InvoiceNo'].str.contains("A")].replace(to_replace=['abcd'],value=['Debt'])

#Catégorisation des annulations de commandes (retours) dans notre nouvelle colonne InvoiceType
df[df['InvoiceNo'].str.contains("C")]=df[df['InvoiceNo'].str.contains("C")].replace(to_replace=['abcd'],value=['Canceled'])

#Catégorisation des autres lignes en ventes 'Sale'
#Pour cela, on extrait les lignes pour lesquels InvoiceNo ne contient pas 'A' ou 'C'
df[-((df['InvoiceNo'].str.contains("A"))&(df['InvoiceNo'].str.contains("C")))]=df[-((df['InvoiceNo'].str.contains("A"))&(df['InvoiceNo'].str.contains("C")))].replace(to_replace=['abcd'],value=['Sale'])

#les différentes valeurs de InvoiceType ainsi que leurs comptes
df.InvoiceType.value_counts()

Sale        531164
Canceled      9288
Debt             3
Name: InvoiceType, dtype: int64

In [ ]:
# 2) Modification des catégories 'Canceled' et 'Sale' en fonction du stockCode et ce à quoi ça correspond

# catégorie Discount
df[df['StockCode']=='D']=df[df['StockCode']=='D'].replace(to_replace=['Canceled','Sale'],value=['Discount','Discount'])

# catégorie Amazon fee
df[df['StockCode']=='AMAZONFEE']=df[df['StockCode']=='AMAZONFEE'].replace(to_replace=['Canceled','Sale'],value=['Amazon fee','Amazon fee'])

# catégorie Commission
df[df['StockCode']=='CRUK']=df[df['StockCode']=='CRUK'].replace(to_replace=['Canceled','Sale'],value=['Commission','Commission'])

# catégorie Sample 
df[df['StockCode']=='S']=df[df['StockCode']=='S'].replace(to_replace=['Canceled','Sale'],value=['Sample','Sample'])

# catégorie Manual order
df[df['StockCode']=='M']=df[df['StockCode']=='M'].replace(to_replace=['Sale','Canceled'],value = ['Manual order','Manual order'])

# catégorie Postage
df[(df['StockCode']=='POST')|(df['StockCode']=='DOT')]=df[(df['StockCode']=='DOT')|(df['StockCode']=='POST')].replace(to_replace=['Sale','Canceled'],value=['Postage','Postage'])

# catégorie Carriage
df[df['StockCode']=='C2']=df[df['StockCode']=='C2'].replace(to_replace=['Sale','Canceled'],value=['Carriage','Carriage'])

# catégorie Bank charges
df[df['StockCode']=='BANK CHARGES']=df[df['StockCode']=='BANK CHARGES'].replace(to_replace=['Sale','Canceled'],value=['Bank charges','Bank charges'])

#les différentes valeurs de InvoiceType ainsi que leurs comptes
df.InvoiceType.value_counts()

Sale            528846
Canceled          8704
Postage           1961
Manual order       571
Carriage           143
Discount            77
Sample              63
Bank charges        37
Amazon fee          34
Commission          16
Debt                 3
Name: InvoiceType, dtype: int64

In [ ]:
# 3) On veut maintenant catégoriser les lignes pour lesquels on a des description qui semblent correspondre à des problèmes
# catégorisation 'Order problem' lorsque la colonne Descirption comporte une valeur appartenant à la liste Description_problems
df[df.Description.isin(Description_problems)] = df[df.Description.isin(Description_problems)].replace(to_replace='Sale',value='Order problem')

#les différentes valeurs de InvoiceType ainsi que leurs comptes
df.InvoiceType.value_counts()

Sale             528244
Canceled           8704
Postage            1961
Order problem       602
Manual order        571
Carriage            143
Discount             77
Sample               63
Bank charges         37
Amazon fee           34
Commission           16
Debt                  3
Name: InvoiceType, dtype: int64

In [ ]:
# 4) catégorisation des lignes pour lesquelles on a un prix == 0 (hors Order problem)
df[df['UnitPrice']==0] = df[df['UnitPrice']==0].replace(to_replace='Sale', value='Gift')

#les différentes valeurs de InvoiceType ainsi que leurs comptes
df.InvoiceType.value_counts()

Sale             527793
Canceled           8704
Postage            1961
Order problem       602
Manual order        571
Gift                451
Carriage            143
Discount             77
Sample               63
Bank charges         37
Amazon fee           34
Commission           16
Debt                  3
Name: InvoiceType, dtype: int64

In [ ]:
#affichage de la table avec les nouvelles colonnes créées
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalAmount,Date,Year,Month,Day,DayOfWeek,DayName,Week,Hour,InvoiceType
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,2010-12-01,2010,12,1,2,Wednesday,48,8,Sale
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,2010-12-01,2010,12,1,2,Wednesday,48,8,Sale
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,2010-12-01,2010,12,1,2,Wednesday,48,8,Sale
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,2010-12-01,2010,12,1,2,Wednesday,48,8,Sale
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,2010-12-01,2010,12,1,2,Wednesday,48,8,Sale


**Création d'une table des ventes**


---


Dans la suite de notre étude, nous utiliserons cette nouvelle table qui ne contient que les ventes de la table initiale

In [ ]:
#Création d'un dataframe comprenant uniquement les ventes (InvoiceType = Sale)
df_sale = df[(df['InvoiceType']=='Sale')]

In [ ]:
#Création d'un dataframe comprenant uniquement les annulations de commandes (=retours) (InvoiceType = Canceled)
df_canceled=df[df['InvoiceType']=='Canceled']
#Nous nous en servirons pas pour le moment, mais cette table peut également faire l'objet d'une étude